In [ ]:
import importlib
import numpy as np
import PyMieScatt as ps
import matplotlib.pyplot as plt
import sphere_difraction as sd

importlib.reload(sd)

## Tasks descriptions

In [ ]:
def SingeLayered():
    lmbd = 0.3 # wave length
    k = 2 * np.pi / lmbd # wave number

    eps = np.array([2.56-0.102j, # permittivities of layers
                    1.0] ) # permittivity of outer space (air)
    
    r = np.array([0.5]) # concentric spheres radiuses

    return r, eps, lmbd

In [ ]:
def TwoLayered():
    c = 3.0e8
    lmbd = 1 # wave length
    k = 2 * np.pi / lmbd # wave number

    eps = np.array([1, 2.56 + 0.102j,  # permittivities of layers
                    1.0]) # permittivity of vacuum (outer space)

    r = np.array([0.5, 0.55]) # concentric spheres radiuses

    return r, eps, lmbd

In [ ]:
def ThreeLayered():
     lmbd = 2 * np.pi / 1000.0 # wave length
     k = 2 * np.pi / lmbd # wave number

     eps = np.array([1, 1.1 + 0.01j, 2.5, # permittivities of layers
                     1.0] ) # permittivity of outer space (air) 

     r = np.array([0.01, 
          0.01 + np.pi/k, 
          0.01 + np.pi/k + 0.5 /k]) # concentric spheres radiuses
     
     return r, eps, lmbd

In [ ]:
def FourLayered():
    lmbd = 2 * np.pi / 1000.0 # wave length
    k = 2 * np.pi / lmbd # wave number

    eps = np.array([1, 2.5, 0.25, # permittivities of layers
                    8.85e-12]) # permittivity of vacuum (outer space)

    r = np.array([0.9/4,
        0.9/3,
        0.9/2,
        0.9]) # concentric spheres radiuses
    
    return r, eps, lmbd


In [ ]:
def SilverDot():
    lmbd = 3e2 # wave length

    eps = np.array([(1.5258 + 1.8878j)**2, # permittivities of layers
                    1.0]) # permittivity of air (outer space)

    r = np.array([5]) # concentric spheres radiuses

    return r, eps, lmbd

### Helper ploting functions

In [ ]:
def plot_field_scaterring(S_th, S_ph):
    num_points = len(S_th)
    y_values = [abs(c) for c in S_ph[:num_points // 2]] + [abs(c) for c in S_th[num_points // 2:]]
    y_values = np.array(y_values)

    theta = np.linspace(0, 2 * np.pi, num_points)

    plt.figure(figsize=(8, 8), dpi=200)

    number = 75
    ax = plt.subplot(111, projection='polar')
    ax.plot(theta, y_values, linestyle='-', linewidth=0.5)
    #ax.set_yticklabels([])

    ax.set_title('Диаграмма рассеяния')

    plt.show()

In [ ]:
def plot_field_distribution(E, limits, r, eps, lmbd, conducting_center = True):
    y_min, y_max, x_min, x_max = limits

    plt.figure(figsize=(7, 6))
    plt.imshow(E/ (1 + E), origin='lower', aspect='auto', cmap="viridis",extent=[y_min, y_max, x_min, x_max])
    plt.colorbar(label='Модуль E')
    if(conducting_center):
        plt.title("Распределение рассеянного электрического поля на проводящем шаре, покрытом слоями диэлектрика")
    else:
        plt.title("Распределение рассеянного электрического поля на шаре, состоящем из слоёв диэлектриков")
    plt.xlabel('Z')
    plt.ylabel('Y')
    
    info_text = f"λ = {lmbd:.5f}\n"
    for i, (radius, epsilon) in enumerate(zip(r, eps), start=1):
        if i == 1:
            continue
        info_text += f"Слой {i-1}: r = {radius:.5f}, ε = {epsilon}\n"

    plt.text(0.05, 0.12, info_text, transform=plt.gca().transAxes,
             fontsize=10, verticalalignment='center', bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.8))
    
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_radar_cross_section(S_theta, k):
    y_values = np.array([abs(c)**2 for c in S_theta][:]) * 4.0 * np.pi * k**2
    num_points = len(y_values) //2

    Right_bound = 180
    theta = np.linspace(0, Right_bound, num_points)
    x_ticks = np.linspace(0, Right_bound, 7)
    y_min, y_max = -25, 25
    y_ticks = np.linspace(y_min,y_max, 6)

    plt.figure(figsize=(8, 6))
    ax = plt.subplot(111)
    ax.grid()
    ax.set_xticks(x_ticks)
    ax.set_yticks(y_ticks)
    ax.set_ylim(y_min,y_max)

    ax.plot(theta, 10 * np.log10(y_values[:num_points][::-1]), linestyle='-')

    #ax.set_title('RCS')
    ax.set_xbound(0, Right_bound)
    ax.set_ylabel('dBm')
    ax.set_xlabel('Theta (deg)')

    plt.show()

## Plots

In [ ]:
importlib.reload(sd)
r, eps, lmbd = TwoLayered()
k = 2 * np.pi / lmbd 
conducting_core = True

E_theta, _ =  sd.calculate_electric_field_far(r, eps, lmbd, conducting_core)

D_e, D_m = sd.calculate_coefficients(k, eps, r, conducting_core)
S_th, S_ph = sd.calculate_S(D_e, D_m)

plot_field_scaterring(S_th, S_ph)
plot_radar_cross_section(S_th, k)
plot_radar_cross_section(S_ph, k)

## 2D Distribution

In [ ]:
importlib.reload(sd)

r, eps, lmbd = ThreeLayered()
limits = [-2 * r[-1], 8 * r[-1], -5 * r[-1], 5 * r[-1]]

vE_r, vE_theta, vE_phi = sd.calculate_electric_field_close_vectorized(r, eps, lmbd, limits, True)
E = abs(np.sqrt(vE_r**2 + vE_theta**2 + vE_phi**2))

In [ ]:
plot_field_distribution(E.T, limits, r, eps, lmbd)

### PyMieScatt test

In [ ]:
r, eps, lmbd = SilverDot()
# параметры частицы
wavelength = lmbd     # nm
diameter = r          # nm
m = eps[:-1]          # комплексный показатель преломления
medium_index = eps[-1]       # воздух

MaxAngle = 360
angles = np.linspace(0, MaxAngle, MaxAngle * 2)
theta, SL, SR, SU = ps.ScatteringFunction(m, wavelength, diameter, maxAngle=MaxAngle, nMedium=medium_index)
#theta, SL, SR, SU = ps.CoreShellScatteringFunction(m[0] * 1e2, m[1], wavelength, diameter[0], diameter[1], maxAngle=MaxAngle, nMedium=medium_index, normed=False)

plt.figure(figsize=(7,6))
ax = plt.subplot(111, polar=True)

#ax.plot(theta, SL, label='S⊥')
ax.plot(theta, SR, label='S||')
ax.set_title("Диаграмма рассеяния Mie (полярные координаты)")
ax.legend(loc="upper right")

plt.show()

## Widgets setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets
from IPython.display import display

def plot_function(RE_eps1=1.0, IM_eps1=0.0, RE_eps2=2.5, IM_eps2=0.0, lmbd = 2 * np.pi / 1000.0, d1 = np.pi/1000, d2 = (np.pi + 0.5)/1000):
    r, eps, _ = ThreeLayered()
    eps[1] = complex(RE_eps1, IM_eps1)
    eps[2] = complex(RE_eps2, IM_eps2)

    #r[1] = r[0] + d1
    #r[2] = r[1] + d2


    limits = [-3 * r[-1], 5 * r[-1], -4 * r[-1], 4 * r[-1]]
    vE_r, vE_theta, vE_phi = sd.calculate_electric_field_close_vectorized(r, eps, lmbd, limits)
    E = abs(np.sqrt(vE_r**2 + vE_theta**2 + vE_phi**2))

    plot_field_distribution(E.T, limits, r, eps, lmbd)


slider_RE_eps1 = widgets.FloatSlider(value=1.0, min=0, max=5, step=0.1)
slider_IM_eps1 = widgets.FloatSlider(value=0.0, min=0, max=0.1, step=0.01)
slider_RE_eps2 = widgets.FloatSlider(value=2.5, min=0, max=5, step=0.1)
slider_IM_eps2 = widgets.FloatSlider(value=0.0, min=0, max=0.1, step=0.01)
#slider_Lambda = widgets.FloatSlider(value=2 * np.pi / 1000.0 , min=0.001, max=0.1, step=0.001)

slider_d1 = widgets.FloatLogSlider(value=0.01, min=0.0, max=0.01, step=0.001 )
slider_d2 = widgets.FloatLogSlider(value=0.01, min=0.0, max=0.01, step=0.001 )

label_EPS = widgets.HTML(value="<b>Layers permittivities</b>")
label_Ds = widgets.HTML(value="<b>Layers thickness</b>")

label_RE_EPS1 = widgets.HTML(value="<b>Re eps1</b>")
label_IM_EPS1 = widgets.HTML(value="<b>Im eps1</b>")
label_RE_EPS2 = widgets.HTML(value="<b>Re eps2</b>")
label_IM_EPS2 = widgets.HTML(value="<b>Im eps2</b>")

column1 = widgets.VBox([label_RE_EPS1, label_IM_EPS1, label_RE_EPS2, label_IM_EPS2])
column2 = widgets.VBox([slider_RE_eps1, slider_IM_eps1, slider_RE_eps2, slider_IM_eps2])#, slider_Lambda])
#column3 = widgets.VBox([label_Ds, slider_d1, slider_d2])


controls = widgets.HBox([column1, column2])#, column3])

# Display the layout
display(controls)

out = widgets.interactive_output(plot_function, {
    'RE_eps1': slider_RE_eps1,
    'IM_eps1': slider_IM_eps1,
    'RE_eps2': slider_RE_eps2,
    'IM_eps2': slider_IM_eps2,
    #'lmbd' : slider_Lambda,
    #'d1': slider_d1,
    #'d2': slider_d2,
})

display(out)